# Deploy your ML model

This notebook focuses on the last step of the data science workflow: deploying your trained ML model. Just to describe the situation. You've followed all the steps of the data science workflow, you've evaluated your model and your are happy with it. Now it is time to deploy your model for users to be able to use it to do predictions.

Deploying your ML model is not about data science, but about data engineering.

We're going to deploy an ML model in three ways:
* a node.js web application where the *back-end* performs the prediction
* a node.js web application where the *front-end* (the browser) performs the prediction
* a python-based web application where the *back-end* performs the prediction

Why three ways? Just that you can choose the way that you like the most.

# Hands-on 5: deploy a pre-trained Keras model using Tensorflow.js

In Hands-on 3, "Univariate linear regression using a Keras/Tensorflow neural network", we've trained a regression model using Keras. We were happy with the model and saved the model, to allow using it elsewhere to make predictions. 

In this hands-on, we're going to deploy the pre-trained Keras model in two ways:
* a web application where the *back-end* performs the prediction. 
* a web application where the *front-end* (the browser) performs the prediction. Note that *training* a machine learning model is CPU-intensive, but *running* is not. So running the trained model in the browser on a light-weight device like a smartphone is not a problem.

Choosing to allocate the prediction intelligence in the back-end or front-end is, of course, an archtitectural design choice, each having its own pro's and con's.

Both ways of deployment use Tensorflow.js. Tensorflow.js is a Javascript based open-source library with which you define, train, and run machine learning models. As we've defined and trained the model using the normal, python-based Tensorflow, we'll be using Tensorflow.js only to *run* the model.

Why do we deploy the Keras model and not the regression based on sklearn which was much easier to perform? By showing how to deploy a Keras model, this method can be used for any Keras model, so also convolutional neural networks, classifications, ....

Yet other possibilities to deploy the pre-trained model are keras.js or ml5.js. ml5.js is an open source, friendly, high level interface to TensorFlow.js. The goal of ml5.js is to make machine learning approachable for a broad audience of artists, creative coders, and students. The library provides access to machine learning algorithms and models in the browser, building on top of TensorFlow.js with no other external dependencies.

**Let's start**

We've saved the model in tensorflow.js format, using ```import tensorflowjs as tfjs; tfjs.converters.save_keras_model(my_regression_model, 'univariateKerasRegression_tfjs_model')```, which allows to directly use it in Tensorflow.js. Another frequently used format is HDF5, the Hierarchical Data Format version 5 (HDF5), is an open source file format that supports large, complex, heterogeneous data. Saving the model in HDF5 format can be done by ```my_regression_model.save('univariateKerasRegression_hf5_model')```. A model saved in HDF5 format (extension .hf5) can be converted to the tensorflow.js format using the tensorflowjs_converter. However, we don't need to do this, as we've saved the model directly into tensorflow.js format.

## Pre-trained Keras model in a Node.js back-end

0. [install node.js, including the so-called "build tools"](https://nodejs.org/en/download/) if you don't have it on your computer
1. open a command prompt in the folder "predict_backend"
2. run ```npm install```.
3. run ```node predict_app.js``` to start the node.js http server
4. open a browser with URL: ```http://localhost:8081/?citysize=20```. '20' means a city size of 200000. The browser should show an expected profit of 195080.9097290039 dollars.

Note, if you get error messages like ```The Node.js native addon module (tfjs_binding.node) can not be found at path```,the so-called "build tools" are not (correctly) installed. The easiest solution on Windows is to remove the folder ```node_modules```, to uninstall node.js and to reinstall node.js, including the build tools. On Mac/Linux, the easiest solution is to remove the folder ```node_modules```, and to uninstall/reinstall the build tools by: ```xcode-select --print-path; sudo rm -r -f /Library/Developer/CommandLineTools; xcode-select --install```. Maybe ```npm install xcode-build-tools``` is also a solution, but I didn't try.
 
Let's have a look at the code of the node.js app in predict_app.js. It's very easy to understand:
* open an HTTP server
* whenever an URL is opened with a query parameter ```citysize```, the ```predictProfit``` function is invoked.
* the function ```predictProfit``` loads the pre-trained Keras/Tensoflow regression model from disk (only once to save resources).
* the ```citysize``` is feature-normalized, a processing step that is is easily forgotten.
* the ```predict``` method is invoked on the loaded regression model and the feature-normalized citysize is passed.
* the predicted value is returned from the ```predict``` method and returned to the browser.

Note that the values of the mean and standard deviation, needed to perform feature normalization, are based on the training set of hands-on 3 'univariate linear regression using a keras/tensorflow NN'. Whenever you change something to the training set, e.g. make it bigger or smaller, the values of the mean and standard deviation are different and the changed values <u>need to be copied to this node.js application</u>, otherwise the predictions will be incorrect!

This example has only one feature (univariate linear regression). If your model has multiple features (multivariate linear regression), the call to ```predict()``` looks as follows:
```
let result = my_regression_model.predict(tf.tensor2d([parseFloat(citysize), parseFloat(citysurface)], [1, 2])).arraySync(); 
```
or alternatively:
```
let result = my_regression_model.predict(tf.tensor2d([[parseFloat(citysize), parseFloat(citysurface)]])).arraySync();
```
You can also pass multiple samples to ```predict()```:
```
let result = my_regression_model.predict(tf.tensor2d([[parseFloat(city1size), parseFloat(city1surface)], [parseFloat(city2size), parseFloat(city2surface)]])).arraySync();
```

## Pre-trained Keras model in a front-end

This web app consists of three parts:
* ```fileserve.js```, an express.js back-end that serves files. Creating your own Express.js file server is very easy and how to do it can be found [here](https://expressjs.com/en/starter/installing.html). To run the file server, you need to
  * run ```npm install``` in the folder of the app to install express.js (if you would start from scratch without a ```package.json```, use ```npm install express --no-save```to install express.js and create a ```package.json```).
  * run ```node fileserve.js``` to start the file server.
* ```main.js```, the application that does the prediction. Please note that the code is almost identical to the code of the Node.js back-end.
* ```index.html```, the front-end. It starts with loading ```tensorflow.js``` and ```main.js```. Then it allows the user to enter the city size, invokes the function that does the prediction, and returns the result.
  * double-clik ```index.html``` to open it in the browser, enter a city size and press ```predict``` to predict the expected profit.

# Overview of popular, python-based options

In the hands-on above, we went from the python world to javascript world by converting the trained model into tensorflow.js format.  However, many data scientists only know python and don't have knowledge about other programming languages. The purpose of this section is to give an overview of currently popular python-based solutions.

1. ***Streamlit***. Streamlit is a <u>server-side web framework</u>, modern (2019) & easy to learn. If you need a very fast and easy way to deploy your ML solution as a web application, choose Streamlit. It has no suppport yet for RESTful API's. Just to give some understanding of the architecture, Streamlit is built on top of Tornado, a Python web framework and asynchronous networking library.
2. ***Django*** and ***Flask***. Django and Flask are full-featured, python-based web frameworks. They do support RESTful API's. Due to their full-featured nature, they come with a learning curve. There many other python-based web frameworks, but Django and Flask are the most well-known.
3. ***Plain python using http and json libraries***. Using the libraries ```json```, ```requests``` and ```urllib.parse```, it is easy to set-up an HTTP server. [This](https://towardsdatascience.com/restful-apis-in-python-121d3763a0e4) is an example.

We're going to do a small demo using Streamlit, just to show how easily it works.

# Hands-on 6: deploy a pre-trained model using Streamlit

In hands-on 4, Linear regression in practice, we've performed a linear regression using the scikit-learn. This is a common method to perform linear regression. In this hands-on, we're going to deploy this model as a web app, using Streamlit. Streamlit is a server-side, python-based web framework that provides a very easy way to deploy your python-based ML solution. 

For every HTTP user request the complete Streamlist python script is run from top to bottom. You want to **train** the model only **once**, as this is time consuming, but **use** the trained model to perform **many** predictions. How to accomplish this? Streamlit has the very useful ```@st.cache``` annotation to accomplish this, which you can put above a function that needs to be performed only once. The first time the function is invoked, it is executed and the function result is also cached. The second time the function is invoked, Streamlit uses the cached result, rather than executing the function again.

1. Install streamlit using ```pip install streamlit``` or ```pip install streamlit --user``` if the first one gives an error message.
2. Add ```C:\Users\<user name>\AppData\Roaming\Python\Python37\Scripts``` to the path is not already done
3. Save the code below in a file called ```streamlit_demo.py``` (in the same folder as where this Jupyter notebook is located)
4. anaconda prompt> ```streamlit run streamlit_demo.py```
5. open web browswer on http://localhost:8501

In [ ]:
##### this code cannot be run as a jupyter notebook
##### save this code to a file called "streamlit_demo.py" and run it using "anaconda prompt> streamlit run streamlit_demo.py"
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
import streamlit as st

@st.cache  # this function is executed for the 1st user request; for subsequent user requests, the cached function result is used
# please note that the code below is identical to the code of hands-on 4
def train_model():
    print("function train_model() is called (to verify the caching behavior pf Streamlit)")

    # read the training set
    data = pd.read_csv("ex1data1.txt", header=None)  # read from dataset into Pandas DataFrame variable
    data.head()  # view first few rows of the data
    X = data.iloc[:, 0]  # read first column; upper case for matrix
    y = data.iloc[:, 1]  # read second column; lower case for vector
    m = len(y)  # number of training samples; lower case for scalar
    X = np.array(X).reshape(-1, 1)  # transform to format that sklearn expects

    # perform the regression
    regr = linear_model.LinearRegression()
    regr.fit(X, y)

    # the theta's
    print('theta_0', regr.intercept_, 'theta_1', regr.coef_)
    
    return regr


st.title('Predicting profit as function of city size')

# train the model. This is done only once due to the Streamlit caching feature
regr = train_model()

# perform prediction
city_size = float(st.text_input("Enter the city size: ", '0'))  # 2nd argument is the value of city_size on 1st rendering of the web app
pred = regr.predict([[city_size]])
print(pred)
st.text('The predicted profit for city size ' + str (city_size * 10000) + ' is ' + str(pred * 10000) + ' dollars.') 
